<a href="https://colab.research.google.com/github/DvirHayat/BAYU-Net/blob/main/UNet_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the ultralytics package
!pip install ultralytics

# Install the roboflow package
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import files, drive
import os
import cv2
import numpy as np


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
drive.mount('/content/drive')
BASE_DATASET_DIR = "/content/drive/MyDrive/dataset_for_Unet/1.0.1"
MODEL_PATH = "/content/drive/MyDrive/dataset_for_Unet/1.0.0/yolo_weights.pt"            #Yolo model weights
FULL_IMAGE_DIR_PATH = "/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint"     #From where take the whole images
YOLO_RESULT_OUTPUT_PATH = "/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_result"   #yolo output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the YOLO model
yolo_model = YOLO(MODEL_PATH)

Each one of the bounding box detected by the yolo need to be processed for the data set.

We need to define the follwoing:
* Fixed size

In [ ]:
FIXED_SIZE = (162,112)

### Image Process function

In [ ]:
def crop_image(orig_img, x1, y1, x2, y2):
    try:
        if orig_img is None:
            raise ValueError("Input image is None")

        height, width = orig_img.shape[:2]

        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        if x1 < 0 or y1 < 0 or x2 > width or y2 > height:
            raise ValueError(f"Crop coordinates out of bounds: ({x1}, {y1}, {x2}, {y2}) for image size ({width}, {height})")

        if x2 <= x1 or y2 <= y1:
            raise ValueError(f"Invalid crop box: x2 <= x1 or y2 <= y1")

        # Perform crop
        cropped = orig_img[y1:y2, x1:x2]
        return cropped

    except Exception as e:
        print(f"[ERROR] crop_image failed: {e}")
        return None


In [ ]:
def center_and_padd_image(image, target_width, target_height):
    try:
        if image is None:
            raise ValueError("Input image is None")

        if not isinstance(image, np.ndarray):
            raise TypeError("Input image must be a NumPy array")

        if len(image.shape) != 3:
            raise ValueError("Input must be a color image with 3 channels")

        img_height, img_width = image.shape[:2]

        if target_width < img_width or target_height < img_height:
                raise ValueError(
                              f"Target size too small for padding: "
                              f"image size = ({img_width}, {img_height}), "
                              f"target size = ({target_width}, {target_height})"
                          )

        # Calculate padding on each side
        x_pad = target_width - img_width
        y_pad = target_height - img_height

        left = x_pad // 2
        right = x_pad - left
        top = y_pad // 2
        bottom = y_pad - top

        padded_image = cv2.copyMakeBorder(
            image,
            top=top,
            bottom=bottom,
            left=left,
            right=right,
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0]  # black padding
        )

        return padded_image

    except Exception as e:
        print(f"[ERROR] center_and_padd_image failed: {e}")
        return None


### Raw Dataset Creation

In [ ]:
# List all png files in the directory
image_name_list = [f for f in os.listdir(FULL_IMAGE_DIR_PATH) if f.lower().endswith('.png') and os.path.isfile(os.path.join(FULL_IMAGE_DIR_PATH, f))]

# Get full paths
full_image_paths = [os.path.join(FULL_IMAGE_DIR_PATH, image_name) for image_name in image_name_list]

# Print filenames
for path in full_image_paths:
    print(path)

/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint/8_6.png
/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint/7_5.png
/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint/10_7.png
/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint/9_5.png
/content/drive/MyDrive/dataset_for_Unet/1.0.1/yolo_endpoint/9_4.png


In [ ]:
 results = yolo_model(full_image_paths)


0: 640x640 5 vertebreas, 261.5ms
1: 640x640 9 vertebreas, 261.5ms
2: 640x640 10 vertebreas, 261.5ms
3: 640x640 9 vertebreas, 261.5ms
4: 640x640 6 vertebreas, 261.5ms
Speed: 8.3ms preprocess, 261.5ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
def process_and_store(prediction, output_name, fixed_size=FIXED_SIZE, output_path=YOLO_RESULT_OUTPUT_PATH):
    boxes = prediction.boxes
    original_image = prediction.orig_img

    for i, xyxy in enumerate(boxes.xyxy):
        try:
            x1, y1, x2, y2 = map(int, xyxy)

            # Crop
            cropped = crop_image(original_image, x1, y1, x2, y2)
            if cropped is None:
                print(f"[WARNING] Skipping box {i}: crop failed")
                continue

            # Center and pad to fixed size
            padded = center_and_padd_image(cropped, fixed_size[0], fixed_size[1])
            if padded is None:
                print(f"[WARNING] Skipping box {i}: padding failed")
                continue

            # Construct unique filename per box
            base_name = os.path.splitext(output_name)[0]
            filename = f"{base_name}_{i}.png"
            full_path = os.path.join(output_path, filename)

            # Save the image
            success = cv2.imwrite(full_path, padded)
            if success:
                print(f"[INFO] Successfully processed box {i}")
            else:
                print(f"[ERROR] Failed to write image for box {i}: {full_path}")

        except Exception as e:
            print(f"[ERROR] Failed on box {i}: {e}")

In [ ]:
for idx, result in enumerate(results):
    output_name = f"result_{idx}"

    try:
        process_and_store(result, output_name=output_name)
        print(f"[INFO] Successfully processed result {idx}")

    except Exception as e:
        print(f"[ERROR] Exception during processing of result {idx}: {e}")


[ERROR] Failed on box 0: name 'crop_image' is not defined
[ERROR] Failed on box 1: name 'crop_image' is not defined
[ERROR] Failed on box 2: name 'crop_image' is not defined
[ERROR] Failed on box 3: name 'crop_image' is not defined
[ERROR] Failed on box 4: name 'crop_image' is not defined
[INFO] Successfully processed result 0
[ERROR] Failed on box 0: name 'crop_image' is not defined
[ERROR] Failed on box 1: name 'crop_image' is not defined
[ERROR] Failed on box 2: name 'crop_image' is not defined
[ERROR] Failed on box 3: name 'crop_image' is not defined
[ERROR] Failed on box 4: name 'crop_image' is not defined
[ERROR] Failed on box 5: name 'crop_image' is not defined
[ERROR] Failed on box 6: name 'crop_image' is not defined
[ERROR] Failed on box 7: name 'crop_image' is not defined
[ERROR] Failed on box 8: name 'crop_image' is not defined
[INFO] Successfully processed result 1
[ERROR] Failed on box 0: name 'crop_image' is not defined
[ERROR] Failed on box 1: name 'crop_image' is not de

Now you need to load the yolo result to roboflow extract the dataset as coco-segmentation and process them into output.

## Dataset Preperation
After the dataset is annonated in roboflow we want to load it and get it ready for the unet to use/ train

In [ ]:
# Load the roboflow dataset it can be chnaged from dataset to another

from roboflow import Roboflow
rf = Roboflow(api_key="nQsUcNQlcVFxKyvIhf5W")
project = rf.workspace("unet-vertebrae-segmentation").project("my-first-project-5xsoq")
version = project.version(5)
dataset = version.download("coco-segmentation")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-5 in coco-segmentation:: 100%|██████████| 156/156 [00:00<00:00, 6969.36it/s]


In [ ]:
import shutil

# Move the dataset into drive
shutil.move("/content/My-First-Project-5", BASE_DATASET_DIR)



'/content/drive/MyDrive/dataset_for_Unet/1.0.1/My-First-Project-5'

In [ ]:
# Store the path of the new data set from roboflow
robo_dataset_dir = os.path.join(BASE_DATASET_DIR, "My-First-Project-5")


The dataset is in coco-segmentation format to get the model trained we need to convert the dataset into png mask format.

In [ ]:
!pip install pycocotools opencv-python pillow

In [ ]:
from pycocotools.coco import COCO
from PIL import Image
from tqdm import tqdm
def coco_to_mask(ann_path, mask_output_dir, class_id=None):
    """
    Converts COCO segmentation annotations into binary mask PNGs.

    Parameters:
    ann_path (str): Path to the COCO JSON annotation file.
    mask_output_dir (str): Directory where the output binary mask images will be saved.
    class_id (int, optional): If specified, only annotations with this category ID will be used for the masks.
    """

    # Create output directory if it doesn't exist
    os.makedirs(mask_output_dir, exist_ok=True)

    # Load the COCO annotations
    coco = COCO(ann_path)

    # Get all image IDs in the dataset
    image_ids = coco.getImgIds()

    for img_id in tqdm(image_ids):
        # Load metadata for the current image
        img_info = coco.loadImgs(img_id)[0]
        height, width = img_info["height"], img_info["width"]
        img_name = img_info["file_name"]

        # Create an empty mask the size of the image
        mask = np.zeros((height, width), dtype=np.uint8)

        # Get annotation IDs for this image
        ann_ids = coco.getAnnIds(imgIds=img_id)

        # Load all annotations
        anns = coco.loadAnns(ann_ids)

        for ann in anns:
            # Skip annotations that don't match the specified class (if provided)
            if class_id and ann['category_id'] != class_id:
                continue

            # Merge the annotation mask with the main mask
            mask = np.maximum(mask, coco.annToMask(ann))

        # Save the binary mask as a grayscale PNG image (0 for background, 255 for object)
        mask_path = os.path.join(mask_output_dir, img_name.replace('.jpg', '.png'))
        Image.fromarray(mask * 255).convert("L").save(mask_path)


In [ ]:
BASE_MASK_DIR = os.path.join(robo_dataset_dir, "mask")

In [ ]:
splits = ["train", "valid", "test"]

for split in splits:
    img_dir = os.path.join(robo_dataset_dir, split)
    ann_path = os.path.join(img_dir, "_annotations.coco.json")  # <-- per folder
    mask_output_dir = os.path.join(BASE_MASK_DIR, split)

    print(f"Processing {split} set...")
    coco_to_mask(
        ann_path=ann_path,
        mask_output_dir=mask_output_dir
    )


Processing train set...
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 116/116 [00:01<00:00, 87.64it/s]


Processing valid set...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


100%|██████████| 16/16 [00:00<00:00, 93.48it/s]


Processing test set...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


100%|██████████| 16/16 [00:00<00:00, 84.64it/s]


At this moment the data is ready to be used in training.